<h1>Proton Therapy Optimization</h1>

<h3>Naive modeling</h3>

The first idea that comes to mind is to make a straight forward mathematical translation of the oral problem description. That would be to set a vector of *i* integer variables *x[i]*, representing the number of proton fractions that should be performed to patient *i*. 

We would then symbolize our input matrix as *c[i,j]*: the benefit of offering *j* proton fractions to patient *i*. While this is a very intuitive model it suffers from a very non linear object function. In the simplest case where we would like to optimize the sum or average BED, the object functions to be maximized would be:

**f = Σ c[i, x[i]] over all i's**

<h3>A better modeling idea</h3>


Instead we would prefer to go with a linear model. For that reason we could select another set of decision variables, that is a binary matrix *x[i,j]* where a value of 1 denotes that *j* fractions should be performed on patient *i*. Obviously we should constraint every rows sum to 1 (a patient will receive 0,1, ... or N fractions).

Our object function can now be expressed as elegandly as:

**f = Σ c[i,j] * x[i,j] over all j's over all i's**

This is obviously as linear as it gets. Lets code it!

In [1]:
# GUROBI must be installed and a licence file must be discoverable in one of the default locations
from gurobipy import *

# Get some additional dependencies
import pandas as pd
import numpy as np

In [2]:
# Read the dataset into a 2d array. benefit[i,j] is the benefit patient i will get from j fractions
data = pd.read_csv('data/PayoffMatrix.txt', delim_whitespace = True)
benefit = data.values

In [3]:
class ProtonOptimizer(object):
    def __init__(self, BED, capacity = 100, model_name = 'proton_therapy'):
        self._BED = BED
        num_patients, max_fractions_per_patient = BED.shape
        self.patients = [i for i in range(num_patients)]
        self.fractions = [j for j in range(max_fractions_per_patient)]
        self.m = Model(model_name)
        
        # Set binary decision variables
        self.x = self.m.addVars(num_patients, max_fractions_per_patient, vtype = GRB.BINARY) 
        
        # Only one choice of fractions per patient is valid
        self.m.addConstrs(quicksum(self.x[i,j] for j in self.fractions) == 1 for i in self.patients)
        
        # We can only perform so many proton therapies per week
        self.m.addConstr(quicksum(
            quicksum(self.x[i,j] * self.fractions[j] for j in self.fractions) 
            for i in self.patients) <= capacity)
        self.m.update()
        
        self.optimum = {}

        
    def _solve(self, debug = False):
        # Set objective
        self.m.setObjective(quicksum(
            self.x[i,j] * self._BED[i,j] for i in self.patients for j in self.fractions),
                            GRB.MAXIMIZE)
        
        self.m.setParam('OutputFlag', debug)
        self.m.update()
        self.m.optimize()
        if self.m.status == GRB.Status.OPTIMAL:
            solution = self.m.getAttr("x", self.x)
            for i in self.patients:
                for j in self.fractions:
                    if(solution[i,j] == 1):
                        self.optimum[i] = j
                        break
        else:
            print("Infeasible model")
    
    def pretty_print(self):
        solution = self.get_optimum()
        for patient, fractions in solution.items():
            print(("Patient " + str(patient) + " should receive " + str(fractions) + " fractions"))
        
    def get_optimum(self):
        if not self.optimum:
            self._solve()
        return self.optimum

In [5]:
import unittest
import random

class ModelingTest(unittest.TestCase):

    def setUp(self):
        """Create a single Proton optimization model."""
        pass
        
    def mock_BED_data(self):
        BED = np.array([[1, 10, 11], [1, 2, 3], [9, 10, 11]])
        max_fractions = BED.shape[1] - 1
        return BED, max_fractions 
    
    def test_infinite_capacity(self):
        """When capacity is infinite, we expect max amount of fractions for all patients"""
        BED, max_fractions = self.mock_BED_data()
        inf_capacity = 10000
        optimizer = ProtonOptimizer(BED, capacity = inf_capacity)
        for patient, fractions in optimizer.get_optimum().items():
            self.assertEqual(fractions, max_fractions)

    def test_capacity_constraint(self):
        """An optimal model surely uses as many fractions as it can"""
        BED, max_fractions = self.mock_BED_data()
        num_patients = BED.shape[0]
        
        # Capacity should be unable to fulfill every patient
        capacity = random.randint(1, max_fractions * num_patients)
        
        optimizer = ProtonOptimizer(BED, capacity = capacity)
        solution = optimizer.get_optimum()
        fractions_used = sum(solution.values())
        self.assertEqual(fractions_used, capacity)

    def tearDown(self):
        """Delete all models."""
        pass

# Run tests
a = ModelingTest()
suite = unittest.TestLoader().loadTestsFromModule(a)
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.010s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>